In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ag-news-classification-dataset/train.csv
/kaggle/input/ag-news-classification-dataset/test.csv


In [2]:
# pip install transformers
# pip  install openpyxl
import torch
from transformers import BertTokenizer
from IPython.display import clear_output
import os
import pandas as pd
import re
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertForSequenceClassification
import gc
import torch
import torch.nn as nn

PRETRAINED_MODEL_NAME = "bert-base-uncased"  
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

df_train = pd.read_csv('/kaggle/input/ag-news-classification-dataset/train.csv')
df_test = pd.read_csv('/kaggle/input/ag-news-classification-dataset/test.csv')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
df_train

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [4]:
df_test

,Class Index,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...
...,...,...,...
7595,1,Around the world,Ukrainian presidential candidate Viktor Yushch...
7596,2,Void is filled with Clement,With the supply of attractive pitching options...
7597,2,Martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...
7598,3,5 of arthritis patients in Singapore take Bext...,SINGAPORE : Doctors in the United States have ...


In [5]:
df_train = df_train.loc[:, ['Title','Description', 'Class Index']]
df_train.columns = ['text_a','text_b', 'label']
df_test = df_test.loc[:, ['Title','Description', 'Class Index']]
df_test.columns = ['text_a','text_b', 'label']

In [6]:
print(df_train[:100])

                                               text_a  \
0   Wall St. Bears Claw Back Into the Black (Reuters)   
1   Carlyle Looks Toward Commercial Aerospace (Reu...   
2     Oil and Economy Cloud Stocks' Outlook (Reuters)   
3   Iraq Halts Oil Exports from Main Southern Pipe...   
4   Oil prices soar to all-time record, posing new...   
..                                                ...   
95  Gene Blocker Turns Monkeys Into Workaholics - ...   
96        Dolphins Too Have Born Socialites (Reuters)   
97  What's in a Name? Well, Matt Is Sexier Than Pa...   
98  UK Scientists Allowed to Clone Human Embryos (...   
99  Russian Alien Spaceship Claims Raise Eyebrows,...   

                                               text_b  label  
0   Reuters - Short-sellers, Wall Street's dwindli...      3  
1   Reuters - Private investment firm Carlyle Grou...      3  
2   Reuters - Soaring crude prices plus worries\ab...      3  
3   Reuters - Authorities have halted oil export\f...      3  


In [7]:
print(df_test[:100])

                                               text_a  \
0                   Fears for T N pension after talks   
1   The Race is On: Second Private Team Sets Launc...   
2       Ky. Company Wins Grant to Study Peptides (AP)   
3       Prediction Unit Helps Forecast Wildfires (AP)   
4         Calif. Aims to Limit Farm-Related Smog (AP)   
..                                                ...   
95  Eriksson doesn #39;t feel any extra pressure f...   
96            Injured Heskey to miss England friendly   
97           Staples Profit Up, to Enter China Market   
98        Delegation Is Delayed Before Reaching Najaf   
99           Consumer Prices Down, Industry Output Up   

                                               text_b  label  
0   Unions representing workers at Turner   Newall...      3  
1   SPACE.com - TORONTO, Canada -- A second\team o...      4  
2   AP - A company founded by a chemistry research...      4  
3   AP - It's barely dawn when Mike Fitzpatrick st...      4  


In [8]:
df_train_train = df_train.loc[:, ['text_a','text_b', 'label']]
print("train樣本數：", len(df_train_train))
df_train_train.to_csv("train.tsv", sep="\t", index=False) 

df_train_test = df_test.loc[:, ['text_a','text_b', 'label']]
print("test樣本數：", len(df_train_test))
df_train_test.to_csv("test.tsv", sep="\t", index=False)

train樣本數： 120000
test樣本數： 7600


In [9]:
from torch.utils.data import Dataset
 
class FakeNewsDataset(Dataset):
    
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"] 
        self.mode = mode
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        #self.label_map = {'1': 0, '2': 1, '3': 2, '4': 3}
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):# 定義回傳一筆訓練 / 測試數據的函式
        
        text_a,text_b,label = self.df.iloc[idx, :].values
        #label_id = self.label_map[label]
        label_tensor = torch.tensor(label-1)
      
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
        
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len

In [10]:
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors,batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0,1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

In [11]:
PRETRAINED_MODEL_NAME = "bert-base-uncased"
NUM_LABELS = 4

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)

trainset = FakeNewsDataset("train", tokenizer=tokenizer)
trainloader = DataLoader(trainset, batch_size=16,collate_fn=create_mini_batch)

testset = FakeNewsDataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=16,collate_fn=create_mini_batch)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

device: cuda:0


In [12]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        for data in dataloader:
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            if predictions is None:  # 將當前 batch 記錄下來
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

In [13]:
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 2  
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        optimizer.zero_grad() # 將參數梯度歸零
        
        # forward pass  
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() # 紀錄當前 batch loss
        
    _, acc = get_predictions(model, trainloader, compute_acc=True)
    _, tacc = get_predictions(model, testloader, compute_acc=True)
    print('[epoch %d] loss: %.3f, acc: %.3f tacc: %.3f' %(epoch + 1, running_loss, acc, tacc))

[epoch 1] loss: 1632.601, acc: 0.946 tacc: 0.937
[epoch 2] loss: 982.228, acc: 0.961 tacc: 0.937


In [14]:
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

EPOCHS = 2 
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        optimizer.zero_grad() # 將參數梯度歸零
        
        # forward pass  
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() # 紀錄當前 batch loss
        
    _, acc = get_predictions(model, trainloader, compute_acc=True)
    _, tacc = get_predictions(model, testloader, compute_acc=True)
    print('[epoch %d] loss: %.3f, acc: %.3f tacc: %.3f' %(epoch + 1, running_loss, acc, tacc))

[epoch 1] loss: 642.987, acc: 0.972 tacc: 0.944
[epoch 2] loss: 547.213, acc: 0.976 tacc: 0.945


In [15]:
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-7)

EPOCHS = 2
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        optimizer.zero_grad() # 將參數梯度歸零
        
        # forward pass  
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() # 紀錄當前 batch loss
        
    _, acc = get_predictions(model, trainloader, compute_acc=True)
    _, tacc = get_predictions(model, testloader, compute_acc=True)
    print('[epoch %d] loss: %.3f, acc: %.3f tacc: %.3f' %(epoch + 1, running_loss, acc, tacc))

[epoch 1] loss: 511.615, acc: 0.978 tacc: 0.947
[epoch 2] loss: 492.659, acc: 0.978 tacc: 0.945
